In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Paglikha ng Kliyente

Sa halimbawang ito, gagamitin natin ang [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) para magkaroon ng access sa LLM.

Ang `model` ay tinukoy bilang `gpt-4o-mini`. Subukang palitan ang model sa ibang model na makikita sa GitHub Models marketplace upang makita ang iba't ibang resulta.

Bilang mabilisang pagsubok, tatakbo tayo ng isang simpleng prompt - `Ano ang kabisera ng France`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Pagpapakilala ng mga Function

Sa halimbawang ito, bibigyan natin ang ahente ng access sa isang tool na isang function na may listahan ng mga available na destinasyon para sa bakasyon at ang kanilang availability.

Maaari mong isipin na ito ay isang senaryo kung saan ang isang travel agent ay maaaring may access sa isang travel database, halimbawa.

Habang sinusuri mo ang halimbawang ito, huwag mag-atubiling subukang magtakda ng mga bagong function at tool na maaaring tawagin ng ahente.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Pagpapakahulugan sa Function Tool
Para magamit ng ahente ang `vacation_destinations` bilang isang `FunctionTool`, kailangan natin itong tukuyin bilang isa.

Magbibigay din tayo ng isang paglalarawan sa tool na makakatulong sa ahente na matukoy kung para saan ginagamit ang tool na ito kaugnay ng gawain na hiniling ng gumagamit.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Pagde-define ng ahente

Ngayon, maaari na nating likhain ang ahente gamit ang code sa ibaba. I-define natin ang `system_message` upang bigyan ang ahente ng mga tagubilin kung paano tulungan ang mga user na makahanap ng mga destinasyon para sa bakasyon.

Itinakda rin natin ang parameter na `reflect_on_tool_use` sa true. Pinapayagan nito ang paggamit ng LLM upang kunin ang sagot mula sa tawag sa tool at ipadala ang sagot gamit ang natural na wika.

Maaari mong itakda ang parameter sa false upang makita ang pagkakaiba.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Pagpapatakbo ng Ahente

Ngayon, maaari nating patakbuhin ang ahente gamit ang paunang mensahe ng user na humihiling na maglakbay sa Tokyo.

Maaari mong palitan ang destinasyon ng lungsod upang makita kung paano tumutugon ang ahente sa pagkakaroon ng lungsod.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa orihinal nitong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
